## <font color='seagreen'>Notebook2 -  Matching Colony growth rate to time of first cell division</font> 

To determine the effects of hypoxia duration on  bacterial resuscitation, time until first division and colony growth rate were extracted from our single cell time-lapse microscopy data. In this notebook we will extract the manually annotated point layer data created in the Napari python viewer to annotate single-cell first division event. 

In [2]:
#used libraries
import pandas as pd
from scipy.ndimage import label as nlabel
from glob import glob
from skimage import io
import numpy as np
from numpy import copy
from nd2reader import ND2Reader
import napari
import seaborn as sns 
import matplotlib.pyplot as plt
from skimage.measure import regionprops_table
from scipy.signal import savgol_filter

***

## Btrack tracking of single colony data

The Btrack napari plugin was used to track single colony growth traces over time (https://www.napari-hub.org/plugins/btrack). Here we load the data and assign them to the corresponding single cell division events. **Note, these steps have already been performed and then mannually corrected using the napari viewer. The resulting data can be found in the folder ./segmented/Rep{i}/{v}/ where i represent the replicate number and v the imaging position.**

Load Btrack trace data.

In [ ]:
trace1,trace2=pd.read_csv(fr'..\Results\all_traces_Rep1.csv'),pd.read_csv(fr'..\Results\all_traces_Rep2.csv')
trace1['rep']=1
trace2['rep']=2
merged_traces=pd.concat([trace1,trace2])
merged_traces
starting_traces=merged_traces[merged_traces['t_x']==0].root.unique()
merged_traces=merged_traces[merged_traces['root'].isin(starting_traces)]

Relabel images based on Btrack traces.

In [ ]:
for i in range(1,3):
    
    segments=glob(fr'../Data/segmented/Rep{i}/*/stacked_img.tiff')
    segments=sorted(segments,key=lambda x: int(x.split('\\')[1].split('.')[0]))
    rep_filt_traces=merged_traces[merged_traces['rep']==i]
    
    v=0
    
    for segment in segments[v:]:
        
        #load segmented untracked images and btrack traces.
        img=io.imread(segment)
        v_traces=rep_filt_traces[rep_filt_traces['v']==v+1]
        
        #loop through the image zstack
        rlabeled_img=np.zeros(img.shape, dtype='uint16')
        for j in range(img.shape[0]):
            zplane=img[j,...]
            labeled_zplane,_=nlabel(zplane)
            #give old labels unique id range
            labeled_zplane[labeled_zplane!=0]+=1000
            
            #identify relevant trace
            z_traces=v_traces[v_traces['t_x']==j]
            z_traces=z_traces.groupby('root').head(1)
            
            #extract coordinates roots
            coordinates_tracking=list(zip(z_traces.root,z_traces.centroid_x,z_traces.centroid_y))
            nonzeros=np.nonzero(labeled_zplane)
            nonzeros_coords=list(zip(nonzeros[0],nonzeros[1]))
            
            #give each colony new labels based on single colony traces
            for coord in coordinates_tracking:
                new_label=coord[0]
                random_label=labeled_zplane[int(round(coord[1])),int(round(coord[2]))]
                
                if new_label!=0 and random_label!=0:
                    labeled_zplane[labeled_zplane==random_label]=new_label
                    rlabeled_img[j,...]=labeled_zplane
                else:
                    distances=[]
                    for nonzeros_coord in nonzeros_coords:
                        distances.append(get_distance((coord[1],coord[2]),nonzeros_coord))
                    index_min = np.argmin(distances)
                    random_label=labeled_zplane[nonzeros_coords[index_min]]
                    labeled_zplane[labeled_zplane==random_label]=new_label
                    rlabeled_img[j,...]=labeled_zplane
                    
        #remove old labels
        rlabeled_img[rlabeled_img>=1000]=0
        #uncomment the io.imsave to overwrite the saved data
        #io.imsave(fr'./segmented/Rep{i}/{v}/rl_stacked_img.tiff',rlabeled_img)
        print(fr'saved Rep{i}/{v}/rl_stacked_img.tiff')
        v+=1

manually check all images and correct if necessary

In [ ]:
files=glob(r'../Data/*.nd2')
rep1=files[0]
rep2=files[1] 
images=ND2Reader(rep1)
images.bundle_axes = 'tyx'
images.iter_axes = 'v'

In [ ]:
#select imaging fov
v=19

#select replicate
i=1

viewer=napari.Viewer()

viewer.add_image(images[v])
viewer.add_labels(io.imread(f'../Data/segmented/Rep{i}/{v}/crl_stacked_img.tiff').astype(int))

In [ ]:
#uncomment the io.imsave to overwrite the saved data
#io.imsave(f'./segmented/Rep{i}/{v}/crl_stacked_img.tiff', viewer.layers['Labels'].data)
viewer.close()
print(v)

***

## Extract growth rates from corrected colony masks over time

Extract area properties from manual corrected tracked colony masks.

In [ ]:
frames=[]
for i in range(1,3):
    
    segments=glob(fr'../Data/segmented/Rep{i}/*/crl_stacked_img.tiff')
    segments=sorted(segments,key=lambda x: int(x.split('\\')[1].split('.')[0]))
    
    v=0
    
    for segment in segments:
        img = io.imread(segment)
        for j in range(img.shape[0]):
            zplane = img[j,...]
            frame=pd.DataFrame(regionprops_table(zplane,properties=['label','area']))
            frame['time'] = j
            frame['v'] = v
            frame['rep']=i
            frames.append(frame)
        v+=1
        
growth_frame=pd.concat(frames)
growth_frame=growth_frame.reset_index()

In [ ]:
growth_frame['condition'] = np.where(growth_frame['v']<= 10, '3.5 Hypoxia', '5 Hypoxia')
growth_frame['id']=growth_frame['label'].astype(str) + '_' + growth_frame['v'].astype(str)+"_"+growth_frame['rep'].astype(str)

Saved resulting growth_frame data.

In [ ]:
#uncomment to overwrite the saved data
#growth_frame.to_csv(fr'../Results/growth_frame.csv')

In [ ]:
ax=sns.lineplot(
    data=growth_frame,
    x="time", y="area", hue="condition")
ax.set_xlabel('time (frames: each frame is 15 mins)')
ax.set_xlim(0,100)# after 100 images become too crowded to analyse in many cases
#uncomment to overwrite the saved figure
#plt.savefig(fr'../Figures/combined_colony_traces_both_replicates.svg')

In [ ]:
ax=sns.lineplot(
    data=growth_frame,
    x="time", y="area", hue="condition", units="id",
    estimator=None, lw=0.15,
)
ax.set_xlabel('time (frames)')
#uncomment to overwrite the saved figure
#plt.savefig(fr'../Figures/single_colony_traces_both_replicates.svg')

Extract growth rates from areas.

The 1st derivative of a linear savitzky golay filter was applied to area increase over time to determine the maximal growth rate of single colonies.

In [ ]:
savgol_filter?

In [ ]:
growth_frame['log_area'] = np.log(growth_frame['area'])
growth_rate_avgs=[]
IDs=[]
Mus=[]
areas=[]
times=[]
lows=[]
tops=[]
nl_areas=[]
for ID in growth_frame['id'].unique():
    area=growth_frame[growth_frame['id']==ID]['log_area'].values
    nl_area=growth_frame[growth_frame['id']==ID]['area'].values
    time=growth_frame[growth_frame['id']==ID]['time'].values*4
    
    if len(area)>1:
        #savgol filter to extract growth rates.
        Mu=savgol_filter(area,28,1,deriv=1,delta=15/60,axis=0,mode='nearest')
        areas.append(area)
        max_ind=np.argmax(Mu)
        Mus.append(Mu)
        low=max_ind-5
        top=max_ind+5
        if low<0:
            low=0
        if top>len(area-1):
            top=len(area)
        growth_rate_avgs.append(Mu[low:top].mean())
        IDs.append(ID)
        times.append(time)
        lows.append(low)
        tops.append(top)
        nl_areas.append(nl_area)

#combine growth rate data.                
avg_growth_frame=pd.DataFrame({'id':IDs,
                               'growth_rate_avgs':growth_rate_avgs,
                                'Mus':Mus,
                                'Area':areas,
                                'time':times,
                                'low':lows,
                                'top':tops,
                                'non log area':nl_areas})

***

## Match colony growth rates with time until first division of single cells

Next, we will match the extracted growth rate data to our single cell first division time points. Each first cell division timepoint is matched to the colony growth rate of which that cell was a part of.

In [ ]:
t2frame=pd.read_csv(fr'../Results/R2_frame.csv')
t2frame['rep']=2
t1frame=pd.read_csv(fr'../Results/R1_frame.csv')
t1frame['rep']=1
earliest_df=pd.concat([t1frame,t2frame])
earliest_df
earliest_df.set_axis(['Unnamed: 0','level_0','index','t','centroid_x','centroid_y','v','condition','time','rep'], axis=1,inplace=True)
tframe=earliest_df[['t','time','centroid_x','centroid_y','v','rep','condition']]


In [ ]:
new_tframes=[]
for i in range(1,3):
    
    segments=glob(fr'../Data/segmented/Rep{i}/*/crl_stacked_img.tiff')
    segments=sorted(segments,key=lambda x: int(x.split('\\')[1].split('.')[0]))
    
    v=0
    
    for segment in segments:
        img = io.imread(segment)
        sel_tframe=tframe[(tframe['v']==f'{v}') & (tframe['rep']==i)]
        labels=[]
        for index,row in sel_tframe.iterrows():
            z=int(float(row['t']))
            y=int(row['centroid_x'])
            x=int(row['centroid_y'])
            label=img[z,y,x]
            labels.append(label)
                
        sel_tframe['labels']=labels
        new_tframes.append(sel_tframe)
        v+=1
    
new_all_tframe=pd.concat(new_tframes)

In [ ]:
#map single colony growth rates to first time point of division cell data in a single dataframe. 
mapping=avg_growth_frame.groupby('id')['growth_rate_avgs'].apply(list).to_dict()
new_all_tframe['id'] = new_all_tframe['labels'].astype(str) + '_' + new_all_tframe['v'].astype(str)+"_"+new_all_tframe['rep'].astype(str)
new_all_tframe['growth_rate'] = new_all_tframe['id'].map(mapping)
new_all_tframe=new_all_tframe.dropna()
new_all_tframe=new_all_tframe.explode(column='growth_rate')
new_all_tframe=new_all_tframe.rename(columns={'t':'earliest division time (frame)','time':'earliest division {hours}'})
new_all_tframe['earliest division {hours}']=new_all_tframe['earliest division {hours}'].astype(float)

In [ ]:
#uncomment to overwrite and save data
#avg_growth_frame.to_csv(fr'../Results/growth_rate.csv')

Plot growth rate vs earliest division timepoints

In [ ]:
new_all_tframe.head(5)

In [ ]:
ax=sns.jointplot(x='earliest division {hours}',y='growth_rate',data=new_all_tframe.reset_index(), hue='condition',alpha=0.75,s=15)
# JointGrid has a convenience function
ax.set_axis_labels('time of first division (hrs)', 'log (average growth rate (dpixel_area/dtime))', fontsize=12)

plt.tight_layout()
#plt.savefig('../Figures/correlation_time_V_average_log_growth_rate.png',dpi=500)

Plot single colony traces and region over which maximum growth rate was determined.

In [ ]:
df=avg_growth_frame
rdf=df.apply(pd.Series.explode).reset_index()
rdf['label']=rdf.id.str.split('_',expand=True)[0]
rdf['v']=rdf.id.str.split('_',expand=True)[1]
rdf['rep']=rdf.id.str.split('_',expand=True)[2]
rdf['conditions']=np.where(rdf['v'].astype(int)<=10,'hypoxia 3.5 days','hypoxia 5 days')

In [ ]:
growth_rate_areas=[]
growth_rate_log_areas=[]
growth_rate_times=[]
for index,row in df.iterrows():
    low=row['low']
    top=row['top']
    growth_rate_areas.append(row['non log area'][low:top])
    growth_rate_log_areas.append(row['Area'][low:top])
    growth_rate_times.append(row['time'][low:top])
df['gr_areas']=growth_rate_areas
df['gr_log_areas']=growth_rate_log_areas
df['gr_times']=growth_rate_times

df2=df[['gr_log_areas','gr_areas','gr_times','id']]
gdf=df2.apply(pd.Series.explode).reset_index()
gdf['label']=gdf.id.str.split('_',expand=True)[0]
gdf['v']=gdf.id.str.split('_',expand=True)[1]
gdf['rep']=gdf.id.str.split('_',expand=True)[2]
gdf['conditions']=np.where(gdf['v'].astype(int)<=10,'hypoxia 3.5 days','hypoxia 5 days')
gdf.head(5)

In [ ]:
rdf['time (hrs)']=rdf['time']*15/60
gdf['time (hrs)']=gdf['gr_times']*15/60

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(40,20))

sns.lineplot(ax=ax[0],data=rdf,x='time (hrs)',y='non log area',hue='conditions',units='id', estimator=None, lw=0.25)
sns.lineplot(ax=ax[0],data=gdf,x='time (hrs)',y='gr_areas',units='id', estimator=None, lw=0.8, color='black',alpha=0.8)
ax[0].set_xlabel('area (pixels)')

sns.lineplot(ax=ax[1],data=rdf,x='time (hrs)',y='Area',hue='conditions',units='id', estimator=None, lw=0.25)
sns.lineplot(ax=ax[1],data=gdf,x='time (hrs)',y='gr_log_areas',units='id', estimator=None, lw=0.8, color='black',alpha=0.8)
ax[1].set_ylim(4.5,14)
ax[1].set_xlabel('log area (pixels)')

plt.savefig('../Figures/annotated_single_colony_growth_traces.svg',dpi=500)

In [ ]:
#uncommnent to save merged data
#new_all_tframe.to_csv('../Results/earliest_div+growth_rate.csv')

Inspect single colony growth determination.

In [ ]:
fig, axes = plt.subplots(nrows=len(df), ncols = 2, figsize=(5,400))
r=0
for index,row in df.iterrows():
    

    tps=row['Mus']
    max_growth=tps.argmax()
    mean_growth=tps[max_growth-12:max_growth+12].mean()
    
    axes[r,0].plot([float(x)*(10/60) for x in range(0,len(row['Mus']))],row['non log area'], color='red')
    axes[r,1].plot([float(x)*(10/60) for x in range(0,len(row['Mus']))],tps, color='black')

    axes[r,0].set_title(f'exponential growth')
    axes[r,1].set_title(f'Mu {mean_growth}')

    r+=1
    
            
plt.tight_layout()
#uncomment to overwrite and save image
#plt.savefig(fr'..\Figures\growth_rate_overview_figures.png')
plt.show()

***

You have completed the final notebook of this analysis section.